In [ ]:
%matplotlib inline
import pandas as pd
pd.options.display.max_columns = 100
from matplotlib import pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
import numpy as np
pd.options.display.max_rows = 100
data = pd.read_json('../input/train.json')
print("\nFirst 8")
data.head(5)
def status(feature):

    print ('Processing '+feature+': ok')
def get_combined_data():
    # reading train data
    train = pd.read_json('../input/train.json')
    
    # reading test data
    test = pd.read_json('../input/test.json')

    # extracting and then removing the targets from the training data 
    targets = train.interest_level
    train.drop('interest_level',1,inplace=True)
    

    # merging train data and test data for future feature engineering
    combined = train.append(test)
    combined.reset_index(inplace=True)
    combined.drop('index',inplace=True,axis=1)
    
    return combined

combined = get_combined_data()
def get_address():
    global combined
    
    #extract street type from each address
    combined['address_type'] = combined['street_address'].map(lambda type: 
                                                              type.split()[-1] if len(type)>1 else "Other")
    
    #map street types to general ones
    street_dict = {
        "Street": "Street",
        "St": "Street",
        "St.": "Street",
        "St..": "Street",
        "St,": "Street",
        "St...": "Street",
        "Avenue": "Avenue",
        "Ave": "Avenue",
        "Ave.": "Avenue",
        "Terrace": "Terrace",
    }
    combined["address_type"] = combined['address_type'].map(street_dict)
get_address()
combined.head(50)
def process_streets():
    
    global combined
    # we clean the street variable
    combined.drop('street_address',axis=1,inplace=True)
    combined.drop('display_address', axis=1, inplace=True)

    # encoding in dummy variable
    street_dummies = pd.get_dummies(combined['address_type'],prefix='address_type')
    combined = pd.concat([combined,street_dummies],axis=1)
    
    # removing the title variable
    combined.drop('address_type',axis=1,inplace=True)
    
    status('streets')
process_streets()
combined.head(50)
def process_features():
    global combined
    
    #replace with len
    combined['features'] = combined['features'].map(lambda d: len(d))
process_features()
combined.head()
def process_description():
    global combined
    
    #replace
    combined['description'] = combined['description'].map(lambda d: len(d.split()))
process_description()
combined = combined.drop("listing_id", 1)
combined = combined.drop("manager_id", 1)
combined = combined.drop("building_id", 1)
from datetime import datetime
from dateutil.parser import parse
combined['created'] = combined['created'].map(
    lambda x: ((datetime.now() - parse(x)).days )
)
combined = combined.drop("photos", 1)
def scale_all_features():
    
    global combined
    combined[combined['longitude']==0] = combined.longitude.median()
    combined[combined.latitude == 0] = combined.latitude.median()
    the_features = list(combined.columns)
    combined[the_features] = combined[the_features].apply(lambda x: x/x.max(), axis=0)
    
    print ('Features scaled successfully !')
scale_all_features()
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.cross_validation import cross_val_score
def compute_score(clf, X, y,scoring='accuracy'):
    xval = cross_val_score(clf, X, y, cv = 5,scoring=scoring)
    return np.mean(xval)
def recover_train_test_target():
    global combined
    
    train0 = pd.read_json('../input/train.json')
    
    targets = train0.interest_level
    targets = targets.map({
        "low": -1,
        "medium": 0,
        "high": 1
    })
    train = combined.ix[0:49351]
    test = combined.ix[49352:]
    
    return train,test,targets
train,test,targets = recover_train_test_target()
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
clf = ExtraTreesClassifier(n_estimators=200)
clf = clf.fit(train, targets)
features = pd.DataFrame()
features['feature'] = train.columns
features['importance'] = clf.feature_importances_
features.sort(['importance'],ascending=False)
model = SelectFromModel(clf, prefit=True)
train_new = model.transform(train)
train_new.shape
test_new = model.transform(test)
test_new.shape
forest = RandomForestClassifier(max_features='sqrt')

parameter_grid = {
                 'max_depth' : [4,5,6,7,8],
                 'n_estimators': list(range(200,300,10)),
                 'criterion': ['gini','entropy']
                 }

cross_validation = StratifiedKFold(targets, n_folds=3)

grid_search = GridSearchCV(forest,
                           param_grid=parameter_grid,
                           cv=cross_validation)

grid_search.fit(train_new, targets)

print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))